In [1]:
!pip install langchain
import langchain

In [2]:
palak_paneer_ingredients = "Palak Paneer for 2: 120g spinach, 150g paneer, 1 onion, 2 tomatoes, 2 chilies, ginger-garlic, 2 tbsp oil, spices (cumin, turmeric, garam masala), cream.​Blanch/blend spinach; sauté onion-tomato masala; mix spinach, add paneer & cream; 30 mins total."

In [3]:
print(palak_paneer_ingredients)

Palak Paneer for 2: 120g spinach, 150g paneer, 1 onion, 2 tomatoes, 2 chilies, ginger-garlic, 2 tbsp oil, spices (cumin, turmeric, garam masala), cream.​Blanch/blend spinach; sauté onion-tomato masala; mix spinach, add paneer & cream; 30 mins total.


In [4]:
#This is called few shot prompting
prompt = f""" Please xtract following information from given text and return a jason object:

ingredients
quantity of an ingredient
instructions

This is the body of text to exctract information from: {palak_paneer_ingredients}
"""

# if prompt="what is recipie for palak paneer" it will be called as zero shot prompting

In [5]:
print(prompt)

 Please xtract following information from given text and return a jason object:

ingredients
quantity of an ingredient
instructions

This is the body of text to exctract information from: Palak Paneer for 2: 120g spinach, 150g paneer, 1 onion, 2 tomatoes, 2 chilies, ginger-garlic, 2 tbsp oil, spices (cumin, turmeric, garam masala), cream.​Blanch/blend spinach; sauté onion-tomato masala; mix spinach, add paneer & cream; 30 mins total.



In [6]:
from openai import OpenAI
import os

In [7]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
response=client.chat.completions.create(
    model='gpt-4.1-nano',
    messages=[{"role":"system","content":"You are a helpful assistant that extracts structured information from text."},
             {"role":"user","content":prompt}],temperature=0,top_p=1
)

In [9]:
print(response)

ChatCompletion(id='chatcmpl-Cjnk0ysDLXykO3an8PnZaVX959NtB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "ingredients": [\n    {\n      "name": "spinach",\n      "quantity": "120g"\n    },\n    {\n      "name": "paneer",\n      "quantity": "150g"\n    },\n    {\n      "name": "onion",\n      "quantity": "1"\n    },\n    {\n      "name": "tomatoes",\n      "quantity": "2"\n    },\n    {\n      "name": "chilies",\n      "quantity": "2"\n    },\n    {\n      "name": "ginger-garlic",\n      "quantity": null\n    },\n    {\n      "name": "oil",\n      "quantity": "2 tbsp"\n    },\n    {\n      "name": "spices",\n      "details": ["cumin", "turmeric", "garam masala"]\n    },\n    {\n      "name": "cream",\n      "quantity": null\n    }\n  ],\n  "instructions": [\n    "Blanch/blend spinach",\n    "Sauté onion-tomato masala",\n    "Mix spinach, add paneer & cream"\n  ]\n}', refusal=None, role='assistant', annotations=[], audio=None, 

In [10]:
print(response.choices[0].message.content)

{
  "ingredients": [
    {
      "name": "spinach",
      "quantity": "120g"
    },
    {
      "name": "paneer",
      "quantity": "150g"
    },
    {
      "name": "onion",
      "quantity": "1"
    },
    {
      "name": "tomatoes",
      "quantity": "2"
    },
    {
      "name": "chilies",
      "quantity": "2"
    },
    {
      "name": "ginger-garlic",
      "quantity": null
    },
    {
      "name": "oil",
      "quantity": "2 tbsp"
    },
    {
      "name": "spices",
      "details": ["cumin", "turmeric", "garam masala"]
    },
    {
      "name": "cream",
      "quantity": null
    }
  ],
  "instructions": [
    "Blanch/blend spinach",
    "Sauté onion-tomato masala",
    "Mix spinach, add paneer & cream"
  ]
}


In [11]:
import json

In [12]:
json.loads(response.choices[0].message.content)

{'ingredients': [{'name': 'spinach', 'quantity': '120g'},
  {'name': 'paneer', 'quantity': '150g'},
  {'name': 'onion', 'quantity': '1'},
  {'name': 'tomatoes', 'quantity': '2'},
  {'name': 'chilies', 'quantity': '2'},
  {'name': 'ginger-garlic', 'quantity': None},
  {'name': 'oil', 'quantity': '2 tbsp'},
  {'name': 'spices', 'details': ['cumin', 'turmeric', 'garam masala']},
  {'name': 'cream', 'quantity': None}],
 'instructions': ['Blanch/blend spinach',
  'Sauté onion-tomato masala',
  'Mix spinach, add paneer & cream']}

In [23]:
get_ingredients_tool = {
    "type": "function",
    "function": {  # Nested under "function" key
        "name": "get_ingredients",
        "description": "Retrieves ingredient, quantity of each ingredient.",
        "parameters": {
            "type": "object",
            "properties": {
                "ingredient": {
                    "type": "string",
                    "description": "Ingredient needed for the recipe. e.g Water, Tofu"
                },
                "Quantity": {
                    "type": "string",
                    "description": "quantity needed. e.g 200ml, 2 cups, 1 tablespoon"
                }
            },
            "required": ["ingredient", "Quantity"],
            "additionalProperties": False
        }
    }
}

In [26]:
response = client.chat.completions.create(
    model='gpt-4o-mini',  # Valid model (gpt-4.1-nano doesn't exist)
    messages=[
        {"role": "system", "content": "You are a helpful assistant that extracts structured information from text."},
        {"role": "user", "content": prompt}
    ],
    temperature=0,
    top_p=1,
    tools=[get_ingredients_tool]  # Use 'tools' not 'functions' [web:182]
)

In [29]:
print(response)

ChatCompletion(id='chatcmpl-CjnxtlXE81lIvbhRtdx6WJuAXKeim', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_XRP4cz7xE1ZM8I6PWxu7mWib', function=Function(arguments='{"ingredient": "spinach", "Quantity": "120g"}', name='get_ingredients'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_sq1oHMfO5rR0J5ljqv66cmA7', function=Function(arguments='{"ingredient": "paneer", "Quantity": "150g"}', name='get_ingredients'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_8XCsf5bBBmdjY8P3IkHw9pKT', function=Function(arguments='{"ingredient": "onion", "Quantity": "1"}', name='get_ingredients'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_2xbAHrSL8LmLMPWAYop81xvW', function=Function(arguments='{"ingredient": "tomatoes", "Quantity": "2"}', name='get_i

In [46]:
print(response.choices[0].message.tool_calls[0].function.arguments)

{"ingredient": "spinach", "Quantity": "120g"}
